### Preparation

In [1]:
#Load libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from module import * 

In [2]:
#Load data in
tickers = ["TSLA", "AAPL", "AMD"]
start = "2010-01-01"
end = "2024-12-31"

df_prices, df_changes = download_stock_price_data(tickers, start, end)

[*********************100%***********************]  3 of 3 completed


### Signal 1: Moving Average Crossover with MACD Indicator

In [3]:
def signal01(prices, short_ma, long_ma, short_macd, long_macd, signal_window_macd):

    #MA Signal
    ma_sig, _ = ma_signal(prices, short_ma, long_ma)

    #MACD Signal
    macd_sig, _ = signal_macd(prices, short_macd, long_macd, signal_window_macd)

    #Combine Signals
    combined = combine_two_subsignals(ma_sig, macd_sig)

    #Return combined signal dataframe
    signals = pd.DataFrame(index=prices.index)
    signals['signal'] = combined
    signals['position_change'] = signals['signal'].diff().fillna(0)

    return signals

In [3]:
signals = signal01(df_prices['AAPL'], 50, 100, 12, 16, 9)

In [4]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(signals)

            signal  position_change
Date                               
2010-01-04     0.0              0.0
2010-01-05     0.0              0.0
2010-01-06     0.0              0.0
2010-01-07     0.0              0.0
2010-01-08     0.0              0.0
2010-01-11     0.0              0.0
2010-01-12     0.0              0.0
2010-01-13     0.0              0.0
2010-01-14     0.0              0.0
2010-01-15     0.0              0.0
2010-01-19     0.0              0.0
2010-01-20     0.0              0.0
2010-01-21     0.0              0.0
2010-01-22     0.0              0.0
2010-01-25     0.0              0.0
2010-01-26     0.0              0.0
2010-01-27     0.0              0.0
2010-01-28     0.0              0.0
2010-01-29     0.0              0.0
2010-02-01     0.0              0.0
2010-02-02     0.0              0.0
2010-02-03     0.0              0.0
2010-02-04     0.0              0.0
2010-02-05     0.0              0.0
2010-02-08     0.0              0.0
2010-02-09     1.0          

### Signal 2: RSI with Bollinger Bands

In [6]:
def signal02(prices, rsi_window_length, lower_rsi_bound, upper_rsi_bound, bollinger_window_length, bollinger_n_stds, persistance):

    #RSI Signal
    rsi_sig, _ = signal_rsi(prices, rsi_window_length, lower_rsi_bound, upper_rsi_bound)

    #Bollinger Signal
    bollinger_sig, _ = signal_bollinger(prices, bollinger_window_length, bollinger_n_stds, persistence=3)

    #Combine Signals
    combined = combine_two_subsignals(rsi_sig, bollinger_sig)

    #Return combined signal dataframe
    signals = pd.DataFrame(index=prices.index)
    signals['signal'] = combined
    signals['position_change'] = signals['signal'].diff().fillna(0)

    return signals

In [3]:
signals = signal02(df_prices['AAPL'], 14, 30, 70, 14, 1, 1)

In [4]:
_, signals['upper_band'], signals['lower_band'] = compute_bollinger_bands(df_prices['AAPL'], window_length=20, num_std=2)

In [5]:
signals['rsi'] = compute_rsi(df_prices['AAPL'], 14)

In [6]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(signals)

            signal  position_change  upper_band  lower_band        rsi
Date                                                                  
2010-01-04     0.0              0.0    3.324184    3.004507  50.000000
2010-01-05     0.0              0.0    3.641026    3.333950  50.000000
2010-01-06     0.0              0.0    3.960672    3.650641  50.000000
2010-01-07     0.0              0.0    4.369763    3.866888  50.000000
2010-01-08     0.0              0.0    4.678998    4.151974  50.000000
2010-01-11     0.0              0.0    4.980784    4.460499  50.000000
2010-01-12     0.0              0.0    5.283184    4.777034  50.000000
2010-01-13     0.0              0.0    5.623296    5.061689  50.000000
2010-01-14     0.0              0.0    5.993955    5.289979  50.000000
2010-01-15     0.0              0.0    6.313512    5.547643  50.000000
2010-01-19     0.0              0.0    6.623490    5.822910  50.000000
2010-01-20     0.0              0.0    6.593414    5.798438  50.000000
2010-0

### Signal 3: Donchian Channel with ADX

In [ ]:
def signal03(prices, adx_window_length, donchian_window_length):
    
    signals = pd.DataFrame(index=prices.index)
    signals['signal'] = 0.0

    adx = compute_adx(prices, adx_window_length)
    donchian_sig, _ = donchian_signals(prices, donchian_window_length)
    donchian_sig = np.asarray(donchian_sig)

    #Custom tradig logic since adx only detects trends but not in which direction -> combine_two_subsignals() function doesn't work
    position = np.zeros(len(prices), dtype=float)
    holding = 0
    for i in range(len(prices)):
        if np.isnan(adx[i]):
            continue
        if holding == 0 and (donchian_sig[i] == 1 or adx[i] > 25):
            holding = 1
        elif holding == 1 and donchian_sig[i] == 0 and adx[i] > 25:
            holding = 0
        position[i] = holding

    signals['signal'] = position
    signals['position_change'] = signals['signal'].diff().fillna(0)
    signals.iloc[0, signals.columns.get_loc('position_change')] = 0

    return signals

In [ ]:
signals = signal03(prices, 14, 20)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(signals)